In [1]:
!pip install pfrl@git+https://github.com/voidful/pfrl.git
!pip install textrl==0.2.13

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-install-_vc9sujt/pfrl_8a275c4d4fac42c9ac0164c10f0dbf2f
  Running command git clone --filter=blob:none --quiet https://github.com/voidful/pfrl.git /tmp/pip-install-_vc9sujt/pfrl_8a275c4d4fac42c9ac0164c10f0dbf2f
  Resolved https://github.com/voidful/pfrl.git to commit 2ad3d51a7a971f3fe7f2711f024be11642990d61
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead, BlenderbotForConditionalGeneration
import logging
import sys
import pfrl
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [3]:
mname= "facebook/blenderbot-400M-distill"
tokenizer = AutoTokenizer.from_pretrained(mname)  
model = BlenderbotForConditionalGeneration.from_pretrained(mname)
model.eval()
model.cuda()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


BlenderbotForConditionalGeneration(
  (model): BlenderbotModel(
    (shared): Embedding(8008, 1280, padding_idx=0)
    (encoder): BlenderbotEncoder(
      (embed_tokens): Embedding(8008, 1280, padding_idx=0)
      (embed_positions): BlenderbotLearnedPositionalEmbedding(128, 1280)
      (layers): ModuleList(
        (0-1): 2 x BlenderbotEncoderLayer(
          (self_attn): BlenderbotAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
     

In [4]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/open_llama/modeling_open_llama.py:42: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)

In [6]:
observaton_list = [['How are you?']]

In [18]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish or len(predicted_list) >= self.env_max_length:
        predicted_text = tokenizer.convert_tokens_to_string(predicted_list[0])
        if sentiment(predicted_text)[0][0]['score'] > 0.5:
          reward = sentiment(predicted_text)[0][0]['score']*100
        else:
          reward = -sentiment(predicted_text)[0][1]['score']*100 -sentiment(predicted_text)[0][2]['score']*100 
      return reward

In [19]:
env = MyRLEnv(model, tokenizer, observation_input=observaton_list,compare_sample=1)
actor = TextRLActor(env,model,tokenizer)
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=100)

In [20]:
actor.predict(observaton_list[0])

[" I'm doing well. How are you?</s>"]

In [21]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='output', 
)

(<textrl.actor.TextPPO at 0x7fefbf913ca0>,
 [{'average_value': 0.060461678,
   'average_entropy': 2.1820645,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': -99.43709671497345},
  {'average_value': -0.100548096,
   'average_entropy': 2.1670096,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': -99.43709671497345},
  {'average_value': -0.053739943,
   'average_entropy': 2.079209,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': -99.43709671497345},
  {'average_value': -0.029316416,
   'average_entropy': 2.1261568,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': -99.43709671497345},
  {'average_value': -0.0039000616,
   'average_entropy': 2.0448022,
   'average_value_loss': nan,
 

In [22]:
agent.load("./output/best")

In [23]:
actor.predict(observaton_list[0])

[" I'm doing well. How are you?</s>"]

In [24]:
!rm -r "/content/output"